In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
import h5py
from tensorflow.keras.preprocessing.sequence import pad_sequences
from pathlib import Path
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, GlobalAveragePooling1D, Softmax
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import random


2024-06-05 15:04:33.647071: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-05 15:04:33.715583: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-05 15:04:34.060628: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-05 15:04:35.213994: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def create_model(input_shape):
    model = Sequential()
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))

    model.add(GlobalAveragePooling1D())
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [3]:
def load_data(file_paths, max_length):
    data = []
    labels = []
    for file_path in file_paths:
        with h5py.File(file_path, 'r') as f:
            vibration_data = f['vibration_data'][:]
            label = 1 if 'good_data' in file_path.parts else 0
            data.append(vibration_data)
            labels.append(label)

    data_padded = pad_sequences(data, maxlen=max_length, padding='post', dtype='float32')
    return np.array(data_padded), np.array(labels)


### balanced dtaset on whole machine and operations

In [4]:
data_root = Path("./data/")
good_file_paths = list((data_root / 'balanced_data' / 'good_data').glob('*.h5'))
bad_file_paths = list((data_root / 'balanced_data' / 'bad_data').glob('*.h5'))
file_paths = good_file_paths + bad_file_paths

In [8]:
max_length = 10000
X, y = load_data(file_paths, max_length)

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

input_shape = (X.shape[1], X.shape[2])
model = create_model(input_shape)
model.fit(X, y, epochs=25, batch_size=32, validation_split=0.2)

X shape: (140, 10000, 3)
y shape: (140,)
Epoch 1/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 239ms/step - accuracy: 0.6939 - loss: 19.8081 - val_accuracy: 1.0000 - val_loss: 2.2601e-21
Epoch 2/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 203ms/step - accuracy: 0.4122 - loss: 21.0687 - val_accuracy: 0.0357 - val_loss: 22.8912
Epoch 3/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 208ms/step - accuracy: 0.6125 - loss: 10.3048 - val_accuracy: 0.0357 - val_loss: 22.7913
Epoch 4/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 208ms/step - accuracy: 0.7132 - loss: 4.5653 - val_accuracy: 1.0000 - val_loss: 0.0022
Epoch 5/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 215ms/step - accuracy: 0.4737 - loss: 4.1513 - val_accuracy: 0.5357 - val_loss: 5.7293
Epoch 6/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 209ms/step - accuracy: 0.8260 - loss: 1.6859 - val_accuracy: 0.4286 - val_loss: 8.6328
Epoch 7/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 210ms/step - accuracy: 0.8646 - loss: 1.5069 - val_accuracy: 0.7857 - val_loss: 0.8248
Epoch 8/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 209ms/step - accuracy: 0.7363

In [10]:
def create_model(input_shape):
    model = Sequential()
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(GlobalAveragePooling1D())
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [11]:

def load_data(file_paths):
    data = []
    labels = []
    max_length = 0
    for file_path in file_paths:
        with h5py.File(file_path, 'r') as f:
            vibration_data = f['vibration_data'][:]
            label = 1 if 'good_data_subset' in file_path.parts else 0
            data.append(vibration_data)
            labels.append(label)
            max_length = max(max_length, len(vibration_data))

    data_padded = pad_sequences(data, maxlen=max_length, padding='post', dtype='float32')
    return np.array(data_padded), np.array(labels), max_length

### Padded data_set with different time length subset of machines

In [12]:
data_root = Path("./data/")
good_file_paths = list((data_root / 'balanced_data_subset' / 'good_data_subset').glob('*.h5'))
bad_file_paths = list((data_root / 'balanced_data_subset' / 'bad_data_subset').glob('*.h5'))
file_paths = good_file_paths + bad_file_paths

In [13]:
random.shuffle(file_paths)

X, y, max_length = load_data(file_paths)

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

input_shape = (max_length, X.shape[2])

model = create_model(input_shape)
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))


X shape: (70, 137392, 3)
y shape: (70,)
Epoch 1/20


/home/fe/asadi/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-06-05 14:48:04.859659: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1125482496 exceeds 10% of free system memory.
2024-06-05 14:48:05.775065: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1125482496 exceeds 10% of free system memory.


1/2 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.5625 - loss: 29.2895

2024-06-05 14:48:08.024458: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 844111872 exceeds 10% of free system memory.
2024-06-05 14:48:08.610100: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 844111872 exceeds 10% of free system memory.


2/2 ━━━━━━━━━━━━━━━━━━━━ 8s 3s/step - accuracy: 0.5208 - loss: 26.2228 - val_accuracy: 0.5000 - val_loss: 2.5579
Epoch 2/20


2024-06-05 14:48:10.814474: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1125482496 exceeds 10% of free system memory.


2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 3s/step - accuracy: 0.4792 - loss: 4.6875 - val_accuracy: 0.5000 - val_loss: 7.4696
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 3s/step - accuracy: 0.5000 - loss: 7.1259 - val_accuracy: 0.8571 - val_loss: 0.4884
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 3s/step - accuracy: 0.6548 - loss: 2.3720 - val_accuracy: 0.5000 - val_loss: 5.6537
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 3s/step - accuracy: 0.4896 - loss: 5.6606 - val_accuracy: 0.8571 - val_loss: 0.4622
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 3s/step - accuracy: 0.7351 - loss: 1.1937 - val_accuracy: 0.5714 - val_loss: 2.5493
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 3s/step - accuracy: 0.5134 - loss: 3.5289 - val_accuracy: 0.8571 - val_loss: 0.9640
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 3s/step - accuracy: 0.7292 - loss: 1.1868 - val_accuracy: 0.5714 - val_loss: 1.0429
Epoch 9/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 3s/step - accuracy: 0.5342 - loss: 1.5995 - val_accuracy: 0.6429 - val_loss: 0.6490
Epoch 10/20
2/2 ━━━━━━━━━━━

### Trimmed data into 12 seconds

In [21]:

def load_data(file_paths):
    data = []
    labels = []
    max_length = 0
    for file_path in file_paths:
        with h5py.File(file_path, 'r') as f:
            vibration_data = f['vibration_data'][:]
            label = 1 if 'good_data_tr2' in file_path.parts else 0
            data.append(vibration_data)
            labels.append(label)
            max_length = max(max_length, len(vibration_data))

    data_padded = pad_sequences(data, maxlen=max_length, padding='post', dtype='float32')
    return np.array(data_padded), np.array(labels), max_length

In [22]:
data_root_tr2 = Path("./data/")
good_file_paths_tr2 = list((data_root_tr2 / 'balanced_data_tr2' / 'good_data_tr2').glob('*.h5'))
bad_file_paths_tr2 = list((data_root_tr2 / 'balanced_data_tr2' / 'bad_data_tr2').glob('*.h5'))
file_paths_tr2 = good_file_paths_tr2 + bad_file_paths_tr2

In [25]:
random.shuffle(file_paths_tr2)

X, y, max_length = load_data(file_paths_tr2)

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

input_shape = (max_length, X.shape[2])

model = create_model(input_shape)
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))


X shape: (394, 24000, 3)
y shape: (394,)
Epoch 1/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 559ms/step - accuracy: 0.5099 - loss: 66.1615 - val_accuracy: 0.5823 - val_loss: 14.7045
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 549ms/step - accuracy: 0.5716 - loss: 10.4187 - val_accuracy: 0.5063 - val_loss: 7.2648
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 591ms/step - accuracy: 0.5936 - loss: 3.4446 - val_accuracy: 0.6962 - val_loss: 2.7563
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 591ms/step - accuracy: 0.7586 - loss: 1.7331 - val_accuracy: 0.6835 - val_loss: 1.1361
Epoch 5/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 607ms/step - accuracy: 0.7694 - loss: 1.1694 - val_accuracy: 0.6835 - val_loss: 1.0463
Epoch 6/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 615ms/step - accuracy: 0.7666 - loss: 1.0528 - val_accuracy: 0.6329 - val_loss: 0.9783
Epoch 7/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 606ms/step - accuracy: 0.7199 - loss: 0.9692 - val_accuracy: 0.6582 - val_loss: 0.8791
Epoch 8/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 598ms/step - accura

#### 1D convolutional layer, a Max Pooling layer with a window size of 2, one fully connected convolutional layer with 64 units, a dropout layer, and an output layer. The activation function used by both convolutional layers is ReLU, whereas the final classification is obtained by applying the SoftMax function

In [2]:
def create_model(input_shape):
    model = Sequential()
    # 1D Convolutional layer
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape))
    # Max Pooling layer
    model.add(MaxPooling1D(pool_size=2))
    # Fully connected convolutional layer
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    # Global average pooling to handle variable input length
    model.add(GlobalAveragePooling1D())
    # Dropout layer
    model.add(Dropout(0.5))
    # Output layer
    model.add(Dense(2, activation='softmax'))  # Assuming binary classification but with softmax

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


In [3]:
def load_data(file_paths):
    data = []
    labels = []
    max_length = 0
    for file_path in file_paths:
        with h5py.File(file_path, 'r') as f:
            vibration_data = f['vibration_data'][:]
            label = 1 if 'good' in file_path.parts else 0
            data.append(vibration_data)
            labels.append(label)
            max_length = max(max_length, len(vibration_data))
    
    # Padding sequences to the maximum length found in the dataset
    data_padded = pad_sequences(data, maxlen=max_length, padding='post', dtype='float32')
    return np.array(data_padded), np.array(labels), max_length


In [4]:
data_root = Path("./data/")
good_file_paths = list((data_root / 'balanced_data' / 'good_data').glob('*.h5'))
bad_file_paths = list((data_root / 'balanced_data' / 'bad_data').glob('*.h5'))
file_paths = good_file_paths + bad_file_paths

# Shuffle file paths before loading data
random.shuffle(file_paths)

# Load data without truncating
X, y, max_length = load_data(file_paths)

# Check the shape of X and y
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

# Split data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Input shape for the model
input_shape = (max_length, X.shape[2])


X shape: (140, 268288, 3)
y shape: (140,)


In [ ]:
# Create and train the model
model = create_model(input_shape)
model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_val, y_val))


Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.9903 - loss: 0.0782 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 17s 3s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 17s 3s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 17s 3s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 17s 3s/step - accuracy: 1.0000 - loss: 0.000